# Peg Keeper Death Spiral Simulation

Here we create a Peg Keeper object for each existing Peg Keeper, and hook it on to an existing StableSwap pool using Curvesim. Then we simulate agents acting against these Peg Keepers and pools based on some input market prices.

The specific input we are studying in this notebook is if the market price for one of the four peg keeper tokens depegs.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from curvesim.pool import get_pool
from src.modules.pegkeeperv1 import PegKeeperV1
from src.modules.aggregator import AggregateStablePrice

In [3]:
USDC_POOL = "0x4DEcE678ceceb27446b35C672dC7d61F30bAD69E"
USDT_POOL = "0x390f3595bca2df7d23783dfd126427cceb997bf4"
USDP_POOL = "0xca978a0528116dda3cba9acd3e68bc6191ca53d0"
TUSD_POOL = "0x34d655069f4cac1547e4c8ca284ffff5ad4a8db0"

# Aggregator Params
SIGMA = 0.001

# PegKeeper Params
CALLER_SHARE = 0.2
CEILING = 25_000_000 # For USDC/USDT

In [4]:
pools = {address:get_pool(address) for address in [USDC_POOL, USDT_POOL, USDP_POOL, TUSD_POOL]}
agg = AggregateStablePrice(pools.values(), SIGMA)

## Drilling into USDC Peg Keeper

In [5]:
pk = PegKeeperV1(pools[USDC_POOL], agg, CALLER_SHARE, CEILING)

In [6]:
import time
pools[USDC_POOL].balances
balance_pegged = pk.pool.balances[pk.I] / pk.precisions[pk.I]
balance_peg = pk.pool.balances[1 - pk.I] / pk.precisions[1 - pk.I]
balance_pegged, balance_peg

(15870994.85236529, 12374530.328004)

In [28]:
pk.update_allowed(balance_peg, balance_pegged, int(time.time()))

True

In [69]:
pk.estimate_caller_profit(int(time.time()))

4.501042205617978e-10

In [50]:
pk.pool.calc_token_amount([0, 0])

2.250521129e-09

In [41]:
pk.calc_change(balance_peg, balance_pegged)

-699292.9048722581

In [70]:
agg.price()

0.9993217014262806

Goal: Implement a function that determines the profit from moving pool price to some target price, then use scipy's optimize to find the most profitable trade. This should account for external market slippage.

Would be cool to then incorporate this into `curvesim` as a `LiquidityLimitedArbitrage`! However, this would likely result in a surprising amount of work, so let's keep things simple for now.

In [ ]:
from scipy.optimize import minimize_scalar, OptimizeResult


def get_trade():
    pass

def profit():
    """
    Calculate profit from trade. Accounts for external slippage
    """
    trade = get_trade()
    pass

def search():
    """
    Find the optimal liquidity-constrained arbitrage.
    """
    p_min, p_max = 0 # FIXME
    res = minimize_scalar(profit, bounds=(p_min, p_max), method='bounded') # TODO which bounds, method?
    if res.success:
        return res.x
    else:
        raise RuntimeError(res.message) # Could make a minimization failed error class

In [ ]:
# TODO Create classes for metrics like in curvesim?
# TODO Determine metrics for PK depeg (i.e., minted crvUSD, burned crvUSD, net unbacked crvUSD, PK profits, liquidity weighted price)
# Definitely want to track p_agg throughout the run.
# Currently assuming LPs don't pull liquidity

In [ ]:
# Generate correlated prices using Cholesky decomposition

# Generate reasonable liquidity curve f(x, sigma)

# Loop through prices:

    # Update oracle prices/aggregator

    # Update external liquidity curve

    # Check update

    # Check arbitrage

    # Update metrics